In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle

Mounted at /content/drive


In [2]:
!pip install pymrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.8 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.4-py3-none-any.whl size=140078 sha256=d4a75251747886b4a039031b673728650c7484d626ad9f6522d72159d0b42a9e
  Stored in directory: /root/.cache/pip/wheels/f7/a2/98/de741b07dff4ebf8f79d17cd943f9d64f77c4fb2f1e08b7816
Successfully built pymrio
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5


In [19]:
#Load Eora26 from 1990 to 2015

import pymrio as py

file_path = '/content/drive/MyDrive/Yannis/Eora26_all/Eora26_{year}_bp.zip'

Eora26_all = {}
for year in range(1990, 2016):
    Eora26_all[year] = py.parse_eora26

In [14]:
Eora26_all[1990]

AttributeError: 'function' object has no attribute 'keys'

In [12]:
# Load Eora26 from 1990 to 2015
import pymrio as py
import pandas as pd

# Assuming you have the necessary columns in the dataset
def extract_data(Eora26, year):
    # Extracting required data from Eora26 for the specific year
    data = []
    for region in Eora26.get_regions():
        for sector in Eora26.get_sectors():
            C_ij = Eora26.get_co2_emissions(region, sector)
            EC_ij = Eora26.get_energy_consumption(region, sector)
            E_ij = Eora26.get_total_energy(region, sector)
            GDP_i = Eora26.get_gdp(region)
            P_i = Eora26.get_population(region)
            data.append({
                'year': year,
                'region': region,
                'sector': sector,
                'carbon_emissions': C_ij,
                'energy_consumption': EC_ij,
                'total_energy': E_ij,
                'gdp': GDP_i,
                'population': P_i
            })
    return pd.DataFrame(data)

# Extract data for all years
all_years_data = []
for year in range(1990, 2016):
    year_data = extract_data(Eora26_all, year)
    all_years_data.append(year_data)

# Combine data from all years into a single DataFrame
df = pd.concat(all_years_data, ignore_index=True)

# Now you can use the previously provided Kaya decomposition function
regions = df['region'].unique()
sectors = df['sector'].unique()
results_df = kaya_decomposition(df, regions, sectors)

# Display the results
import ace_tools as tools; tools.display_dataframe_to_user(name="Kaya Decomposition Results", dataframe=results_df)


AttributeError: 'dict' object has no attribute 'get_regions'

In [ ]:
def kaya_decomposition(df, regions, sectors):
    results = []
    for region in regions:
        for sector in sectors:
            # Extracting relevant data for each region and sector
            C_ij = df.loc[(df['region'] == region) & (df['sector'] == sector), 'carbon_emissions'].values[0]
            EC_ij = df.loc[(df['region'] == region) & (df['sector'] == sector), 'energy_consumption'].values[0]
            E_ij = df.loc[(df['region'] == region) & (df['sector'] == sector), 'total_energy'].values[0]
            GDP_i = df.loc[df['region'] == region, 'gdp'].values[0]
            P_i = df.loc[df['region'] == region, 'population'].values[0]

            # Calculating the components of the Kaya equation
            C_EC = C_ij / EC_ij
            EC_E = EC_ij / E_ij
            E_GDP = E_ij / GDP_i
            GDP_P = GDP_i / P_i

            # Storing the results
            results.append({
                'region': region,
                'sector': sector,
                'C_EC': C_EC,
                'EC_E': EC_E,
                'E_GDP': E_GDP,
                'GDP_P': GDP_P,
                'population': P_i
            })

    return pd.DataFrame(results)


In [ ]:
regions = df['region'].unique()
sectors = df['sector'].unique()
results_df = kaya_decomposition(df, regions, sectors)
print(results_df)


In [ ]:
results_df.set_index(['region', 'sector']).plot(kind='bar', stacked=True)
plt.title('Kaya Decomposition Results by Region and Sector')
plt.ylabel('Contribution to Carbon Emissions')
plt.show()


In [21]:
# Load Eora26 from 1990 to 2015
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import pymrio as py

# Initialize an empty dictionary to store the Eora26 data for each year
Eora26_all = {}

# Load the Eora26 data for each year
for year in range(1990, 2016):
    file_path = f'/content/drive/MyDrive/Yannis/Eora26_all/Eora26_{year}_bp.zip'
    Eora26 = Eora26_all[year]

# Function to extract data from Eora26 for a specific year
def extract_data(Eora26, year):
    data = []
    for region in Eora26.get_regions():
        for sector in Eora26.get_sectors():
            C_ij = Eora26.satellite.F_Y.loc[region, sector]  # Example for CO2 emissions
            EC_ij = Eora26.satellite.F_Y.loc[region, sector]  # Example for energy consumption
            E_ij = Eora26.satellite.F_Y.loc[region, sector]   # Example for total energy
            GDP_i = Eora26.satellite.Y.loc[region]            # Example for GDP
            P_i = Eora26.satellite.Y.loc[region]              # Example for population

            # Add the extracted data to the list
            data.append({
                'year': year,
                'region': region,
                'sector': sector,
                'carbon_emissions': C_ij,
                'energy_consumption': EC_ij,
                'total_energy': E_ij,
                'gdp': GDP_i,
                'population': P_i
            })
    return pd.DataFrame(data)

# Extract data for all years
all_years_data = []
for year in range(1990, 2016):
    year_data = extract_data(Eora26_all[year], year)  # Pass the correct Eora data for the year
    all_years_data.append(year_data)

# Combine data from all years into a single DataFrame
df = pd.concat(all_years_data, ignore_index=True)

# Assuming kaya_decomposition is correctly defined somewhere
# You need to define or import the kaya_decomposition function here
# results_df = kaya_decomposition(df, regions, sectors)

# Display the results (This line won't work without the kaya_decomposition function)
# import ace_tools as tools; tools.display_dataframe_to_user(name="Kaya Decomposition Results", dataframe=results_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: 1990